In [33]:
if(!require("tidyverse")) install.packages("tidyverse")
if(!require("caret")) install.packages("caret")
if(!require("gbm")) install.packages("gbm")
if(!require("parallel")) install.packages("parellel")
if(!require("doParallel")) install.packages("doParallel")
if(!require("plyr")) install.packages("plyr")
if(!require("e1071")) install.packages("e1071")

require("tidyverse")
require("caret")
require("parallel")
require("doParallel")
require("plyr")
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

Loading required package: e1071



In [35]:
combina <-c("filter128_pool2","filter128_pool4","filter64_pool4")
for (com in combina){
  load(paste0("Conv_Encod_Flat_",com,".RData"))
  training <- data.frame(y=as.factor(y_radio_train),predict_enc_train)
  nzv <- nearZeroVar(training)
  training <- training[, -nzv]
  #test <- data.frame(y=as.factor(y_radio_test),predict_enc_test)
  training$y <- revalue(training$y, c("0"="effusion", "1"="normal"))
  #test$y <- revalue(test$y, c("0"="effusion", "1"="normal"))

  control <- trainControl(
    method = "repeatedcv",
    number = 5,
    repeats = 3,
    classProbs = TRUE,
    allowParallel = TRUE,
    summaryFunction = twoClassSummary
  )

  grid <- expand.grid(
    .interaction.depth = 1,
    .n.trees = c(500, 1500, 3000),
    .shrinkage = .01,
    .n.minobsinnode = 10
  )

  metric <- "Accuracy"

  stump_adaboost <- train(y ~ .,
    data = training,
    method = "gbm",
    bag.fraction = 0.5,
    distribution = "adaboost",
    trControl = control,
    tuneGrid = grid,
    verbose = FALSE,
    metric = metric
  )

  save(stump_adaboost,file=paste0("stump_adaboost",com,".Rdata"))
}



Warning message in train.default(x, y, weights = w, ...):
“The metric "Accuracy" was not in the result set. ROC will be used instead.”
Warning message in train.default(x, y, weights = w, ...):
“The metric "Accuracy" was not in the result set. ROC will be used instead.”
Warning message in train.default(x, y, weights = w, ...):
“The metric "Accuracy" was not in the result set. ROC will be used instead.”
